Structure:

1. Introducing the EDC Pangeo Cloud Platform and Dask
2. Accessing the EarthCODE Open Science Catalog Programatically via STAC API
3. Accessing Data on the OSC - Introducing Zarr/Chunking
4. Introducing Xarray and Distributed Computing with Dask
5. Example
6. Saving your work and Publishing to the EarthCODE Catalog



TODO's: 
- [ ] Make repo
- [ ] Try to load data in a more stac friendly way - odc.load / stackstac
- [ ] Tidy up and correct markdown/more comments
- [ ] References need to be good and to make it a point during the talk
- [ ] Publishing - go end-to-end
- [ ] Fix imports
- [ ] Read in saved polygon from file rather than pasted
- [ ] Dask needs to be properly instantiated
- [ ] All needs to run on EDC
- [ ] Projections need to be fixed (see hv plot with two plots at the bottom)

> Cite As
Alonso, Lazaro, Gans, Fabian, Karasante, Ilektra, Ahuja, Akanksha, Prapas, Ioannis, Kondylatos, Spyros, Papoutsis, Ioannis, Panagiotou, Eleannna, Michail, Dimitrios, Cremer, Felix, Weber, Ulrich, & Carvalhais, Nuno. (2022). SeasFire Cube: A Global Dataset for Seasonal Fire Modeling in the Earth System (0.4) [Data set]. Zenodo. https://doi.org/10.5281/zenodo.13834057


In [ ]:
import stackstac
import xarray
import pystac
import dask.distributed
import rasterio
from odc.stac import stac_load
import odc
import os
from odc.stac import configure_rio
from pystac.extensions.datacube import DatacubeExtension
from odc.stac import configure_s3_access, stac_load
from pystac.extensions.storage import StorageExtension
from datetime import datetime
from pathlib import Path

import geopandas as gpd
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import requests
import xarray as xr
from ipyleaflet import Map, Polygon
from shapely import geometry
from shapely.geometry import shape, box
from ipyleaflet import Map, GeoJSON

from pystac_client import Client as pystac_client
from odc.stac import configure_rio, stac_load



In [ ]:
epsg = 4326
# make the other one use this way
# 1. Your GeoJSON feature as a Python dict
feature = {
  "type": "Feature",
  "properties": {
    "label": "Polygon 1",
    "color": "red"
  },
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [
        [
          -7.661233095720918,
          41.371255582375056
        ],
        [
          -7.655930125371327,
          40.98784631510793
        ],
        [
          -6.819489589712093,
          41.0006005650834
        ],
        [
          -6.8371844752309325,
          41.38919212862254
        ],
        [
          -7.661233095720918,
          41.371255582375056
        ]
      ]
    ]
  }
}

poly = shape(feature["geometry"])
bbox = list(poly.bounds)
center = ((bbox[1] + bbox[3]) / 2.0, (bbox[0] + bbox[2]) / 2.0)
m = Map(center=center, zoom=10)
# Create polygon from lists of points
polygon = Polygon(
    locations=[
        (bbox[1], bbox[0]),
        (bbox[3], bbox[0]),
        (bbox[3], bbox[2]),
        (bbox[1], bbox[2]),
    ],
    color="green",
    fill_color="green",
)

# Add the polygon to the map
m.add(polygon)
m


# Introducing the EDC Pangeo Cloud Platform

In [ ]:
from distributed.client import _global_clients
for client in list(_global_clients.values()):
    client.close()


In [ ]:
# xcube --> open with xcube

In [ ]:
# client.close()

In [ ]:
from distributed import LocalCluster
cluster = LocalCluster()
client = cluster.get_client()
client

### What is [Dask](https://docs.dask.org/) ?

**Dask** scales the existing Python ecosystem: with very or no changes in your code, you can speed-up computation using Dask or process bigger than memory datasets.

- Dask is a flexible library for parallel computing in Python.
- It is widely used for handling large and complex Earth Science datasets and speed up science.
- Dask is powerful, scalable and flexible. It is the leading platform today for data analytics at scale.
- It scales natively to clusters, cloud, HPC and bridges prototyping up to production.
- The strength of Dask is that is scales and accelerates the existing Python ecosystem e.g. Numpy, Pandas and Scikit-learn with few effort from end-users.

It is interesting to note that at first, [Dask has been created to handle data that is larger than memory, on a single computer](https://coiled.io/blog/history-dask/). It then was extended with Distributed to compute data in parallel over clusters of computers.

### What is a Dask Distributed cluster ?

A Dask Distributed cluster is made of two main components:

- a Scheduler, responsible for handling computations graph and distributing tasks to Workers.
- One or several (up to 1000s) Workers, computing individual tasks and storing results and data into distributed memory (RAM and/or worker's local disk).

A user usually needs __Client__ and __Cluster__ objects as shown below to use Dask Distributed.    

![Dask Distributed Cluster](https://user-images.githubusercontent.com/306380/66413985-27111600-e9be-11e9-9995-8f418ff48f8a.png)

#### Where can we deploy a Dask distributed cluster?


[Dask distributed clusters can be deployed on your laptop or on distributed infrastructures (Cloud, HPC centers, Hadoop, etc.).] - (https://docs.dask.org/en/stable/deploying.html)  Dask distributed `Cluster` object is responsible of deploying and scaling a Dask Cluster on the underlying resources.

EDC has one such deployment


![Dask Cluster deployment](https://docs.dask.org/en/stable/_images/dask-cluster-manager.svg)

:::{tip}
A Dask `Cluster` can be created on a single machine (for instance your laptop) e.g. there is no need to have dedicated computational resources. However, speedup will only be limited to your single machine resources if you do not have dedicated computational resources!
:::

## Scaling your Computation using Dask Gateway.

For this workshop, according to the Pangeo EDC deployment, you will learn how to use Dask Gateway to manage Dask clusters over Kubernetes, allowing to run our data analysis in parallel e.g. distribute tasks across several workers.

Lets set up your Dask cluster through Dask Gateway.  
As Dask Gateway is configured by default on this ifnrastructure, you just need to execute the following cells.

In [ ]:
# from dask.distributed import Client

# client = Client()   # create a local dask cluster on the local machine.
# client

Inspecting the `Cluster Info` section above gives us information about the created cluster: we have 2 or 4 workers and the same number of threads (e.g. 1 thread per worker). 

<div class="alert alert-warning">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Go further</b>
    <br>
    <ul>
        <li> You can also create a local cluster with the `LocalCluster` constructor and use `n_workers` 
        and `threads_per_worker` to manually specify the number of processes and threads you want to use. 
        For instance, we could use `n_workers=2` and `threads_per_worker=2`.  </li>
        <li> This is sometimes preferable (in terms of performance), or when you run this tutorial on your PC, 
        you can avoid dask to use all your resources you have on your PC!  </li>
    </ul>
</div>

### Dask distributed Client
 
The Dask distributed `Client` is what allows you to interact with Dask distributed Clusters. When using Dask distributed, you always need to create a `Client` object. Once a `Client` has been created, it will be used by default by each call to a Dask API, even if you do not explicitly use it.

No matter the Dask API (e.g. Arrays, Dataframes, Delayed, Futures, etc.) that you use, under the hood, Dask will create a Directed Acyclic Graph (DAG) of tasks by analysing the code. Client will be responsible to submit this DAG to the Scheduler along with the final result you want to compute. The Client will also gather results from the Workers, and aggregate it back in its underlying Python process.

Using `Client()` function with no argument, you will create a local Dask cluster with a number of workers and threads per worker corresponding to the number of cores in the 'local' machine. Here, during the workshop, we are running this notebook in Pangeo EOSC cloud deployment, so the 'local' machine is the jupyterlab you are using at the Cloud, and the number of cores is the number of cores on the cloud computing resources you've been given (not on your laptop).

### Dask Dashboard

Dask comes with a really handy interface: the Dask Dashboard. It is a web interface that you can open in a separated tab of your browser (but not with he link above, you've got to use Jupyterlabs proxy: https://pangeo-foss4g.vm.fedcloud.eu/jupyterhub/user/_yourusername_/proxy/8787/status).

We will learn here how to use it through [dask jupyterlab extension](https://github.com/dask/dask-labextension).  

To use Dask Dashboard through jupyterlab extension on Pangeo EDC infrastructure,
you will just need too look at the html link you have for your jupyterlab, and Dask dashboard port number, as highlighted in the figure below.

![Dash Board link](./static/dashboardlink.png)
![Dash lab](./static/dasklab.png)

Then click the orange icon indicated in the above figure, and type 'your' dashboard link (normally, you just need to replace 'todaka' to 'your username').  


You can click several buttons indicated with blue arrows in above figures, then drag and drop to place them as your convenience.  

![Example dask lab](./static/exampledasklab.png)


It's really helpfull to understand your computation and how it is distributed.

# Accessing the EarthCODE Catalog

This section introduces [STAC](https://stacspec.org/), the SpatioTemporal Asset Catalog. STAC provides a standardized way to structure metadata about spatialotemporal data. The STAC community are building APIs and tools on top of this structure to make working with spatiotemporal data easier.


Users of STAC will interact most often with Collections and Items (there's also Catalogs, which group together collections). A Collection is just a collection of items, plus some additional metadata like the license and summaries of what's available on each item. You can view available collections on the EarthCODE catalog with



In [ ]:
cat = pystac_client.open("https://catalog.osc.earthcode.eox.at/")
cat

Now let's search and access the same data we were just looking at, SeasFire https://opensciencedata.esa.int/products/seasfire-cube/collection


In the examples we've seen so far, we've just been given a STAC item. How do you find the items you want in the first place? That's where a STAC API comes in.

A STAC API is some web service that accepts queries and returns STAC objects. The ability to handle queries is what differentiates a STAC API from a static STAC catalog, where items are just present on some file system.



In [ ]:
# CODE TO SEARCH AND ACCESS SEASFIRE

The collection points to another collection, which contains the actual data. The EarthCODE STAC extension describes some metadata that enrich the STAC collection https://github.com/stac-extensions/osc.

![img.png](./EarthCODE-STAC.png)

# Accessing Data from EarthCODE

Loading the actual data

The SeasFire Cube is a scientific datacube for seasonal fire forecasting around the globe. It has been created for the SeasFire project, that adresses 'Earth System Deep Learning for Seasonal Fire Forecasting' and is funded by the European Space Agency (ESA)  in the context of ESA Future EO-1 Science for Society Call. It contains almost 20 years of data (2001-2021) in an 8-days time resolution and 0.25 degrees grid resolution. It has a diverse range of seasonal fire drivers. It expands from atmospheric and climatological ones to vegetation variables, socioeconomic and the target variables related to wildfires such as burned areas, fire radiative power, and wildfire-related CO2 emissions.

In [ ]:
seasfire = pystac.read_file(
    "https://s3.waw4-1.cloudferro.com/EarthCODE/Catalogs/seasfire/seasfire-cube_v0.4/catalog.json"
)

for item in seasfire.get_items():
    print(item)

seasfire
# https://s3.waw4-1.cloudferro.com/EarthCODE/Catalogs/seasfire/seasfire-cube_v0.4/seasfire-cube-v.0.4/seasfire-cube-v.0.4.json

## Context

When dealing with large data files or collections, it's often impossible to load all the data you want to analyze into a single computer's RAM at once. This is a situation where the Pangeo ecosystem can help you a lot. Xarray offers the possibility to work lazily on data __chunks__, which means pieces of an entire dataset. By reading a dataset in __chunks__ we can process our data piece by piece on a single computer and even on a distributed computing cluster using Dask (Cloud or HPC for instance).

How we will process these 'chunks' in a parallel environment will be discussed in [dask_introduction](./dask_introduction.ipynb). The concept of __chunk__ will be explained here.

When we process our data piece by piece, it's easier to have our input or ouput data also saved in __chunks__. [Zarr](https://zarr.readthedocs.io/en/stable/) is the reference library in the Pangeo ecosystem to save our Xarray multidimentional datasets in __chunks__.

[Zarr](https://zarr.readthedocs.io/en/stable/) is not the only file format which uses __chunk__. We will also be using [kerchunk library](https://fsspec.github.io/kerchunk/) in this notebook to build a virtual __chunked__ dataset based on NetCDF files, and show how it optimizes the access and analysis of large datasets.

The analysis is very similar to what we have done in previous episodes, however we will use data on a global coverage and not only on a small geographical area (e.g. Lombardia).

### Data

In this episode, we will be using Global Long Term Statistics (1999-2019) products provided by the [Copernicus Global Land Service](https://land.copernicus.eu/global/index.html) and access them through [S3-comptabile storage](https://en.wikipedia.org/wiki/Amazon_S3) ([OpenStack Object Storage "Swift"](https://wiki.openstack.org/wiki/Swift)) with a data catalog we have created and made publicly available.

In [ ]:
seasfire_cube = seasfire.get_item("seasfire-cube-v.0.4")
seasfire_cube

## Assets
STAC is a metadata standard. It doesn't really deal with data files directly. Instead, it links to the data files under the "assets" property.

---

The STAC catalog contains a collection for each Zarr store and there are collection-level assets that point to the location of the Zarr store. There are no items at all in this setup.

In this scenario any STAC metadata exists purely for discovery and cannot be used for filtering or subsetting (see Future Work for more on that). To search the STAC catalog to find collections of interest you will use the Collection Search API Extension. Depending on the level of metadata that has been provided in the STAC catalog you can search by the name of the collection and possibly by the variables – exposed via the Data Cube Extension.

Read straight to xarray
Once you have found the collection of interest, the best approach for accessing the data is to construct the lazily-loaded data cube in xarray (or an xarray.DataTree if the Zarr store has more than one group) and filter from there.

To do this you can use the zarr backend directly or you can use the stac backend to streamline even more. The stac backend is mostly useful if the STAC collection uses the xarray extension.

Constructing the lazy data cube is likely to be very fast if there is a consolidated metadata file OR the data is in Zarr-3 and the Zarr metadata fetch is highly parallelized (read more).



## What is a __chunk__

If you look carefully to `LTS`, each Data Variable is a `dask.array` with a chunk size of `(15680, 40320)`. So basically accessing one data variable would load arrays of dimensions `(15680, 40320)` into the computer's RAM. You can see this information and more details by clicking the icon as indicated in the image below.

![Dask.array](../figures/datasize.png)

When you open one or several netCDF files with `open_mdfataset`, by default, the chunks correspond to the entire size of the variable data array read from each file. When you need to analyze large files, a computer's memory may not be sufficient anymore (see in this example, 2.36GiB for one chunk!).

This is where understanding and using chunking correctly comes into play.


__Chunking__ is splitting a dataset into small pieces. 

Original dataset is in one piece,  
![Dask.array](../figures/notchunked.png)

and we split it into several smaller pieces.  
![Dask.array](../figures/chunked.png)

We split it into pieces so that we can process our data block by block or __chunk__ by __chunk__.

In our case, for the moment, the dataset is composed of several files, so already several pieces (or just one in the example above), and Xarray just creates one chunk for each variable of each file.

Zarr format main characteristics are the following:

- Every chunk of a Zarr dataset is stored as a single file (see x.y files in `ls -al test.zarr/nobs`)
- Each Data array in a Zarr dataset has a two unique files containing metadata:
  - .zattrs for dataset or dataarray general metadatas
  - .zarray indicating how the dataarray is chunked, and where to find them on disk or other storage.
  
Zarr can be considered as an Analysis Ready, cloud optimized data (ARCO) file format, discussed in [data_discovery](./data_discovery.ipynb) section.

In [ ]:
http_url = seasfire_cube.assets["data"].href.replace(
    "s3://",
    f"{seasfire_cube.properties['storage:schemes'][seasfire_cube.assets['data'].extra_fields['storage:refs'][0]]['platform'].rstrip('/')}/",
)

ds = xarray.open_dataset(
	http_url,
	engine='zarr',
    chunks={},
	consolidated=True
	# storage_options = {'token': 'anon'}
)
ds

## What is xarray?

Xarray introduces labels in the form of dimensions, coordinates and attributes on top of raw NumPy-like multi-dimensional arrays, which allows for a more intuitive, more concise, and less error-prone developer experience.

### How is xarray structured?

Xarray has two core data structures, which build upon and extend the core strengths of NumPy and Pandas libraries. Both data structures are fundamentally N-dimensional:

- [DataArray](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html#xarray.DataArray) is the implementation of a labeled, N-dimensional array. It is an N-D generalization of a Pandas.Series. The name DataArray itself is borrowed from [Fernando Perez’s datarray project](http://fperez.org/py4science/datarray/), which prototyped a similar data structure.

- [Dataset](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html#xarray.Dataset) is a multi-dimensional, in-memory array database. It is a dict-like container of DataArray objects aligned along any number of shared dimensions, and serves a similar purpose in xarray as the pandas.DataFrame.



Data can be read from online sources, as in the example above where we just Loaded those into memory in parallel using Dask



## Accessing Coordinates and Data Variables 
DataArray, within Datasets, can be accessed through:
- the dot notation like Dataset.NameofVariable  
- or using square brackets, like Dataset['NameofVariable'] (NameofVariable needs to be a string so use quotes or double quotes)

In [ ]:
ds.latitude

In [ ]:
ds.lai

ds['lai'] is a one-dimensional `xarray.DataArray` with dates of type `datetime64[ns]`

In [ ]:
ds['lai']

In [ ]:
ds['lai'].attrs

### Xarray and Memory usage

Once a Data Array|Set is opened, xarray loads into memory only the coordinates and all the metadata needed to describe it.
The underlying data, the component written into the datastore, are loaded into memory as a NumPy array, only once directly accessed; once in there, it will be kept to avoid re-readings.
This brings the fact that it is good practice to have a look to the size of the data before accessing it. A classical mistake is to try loading arrays bigger than the memory with the obvious result of killing a notebook Kernel or Python process.
If the dataset does not fit in the available memory, then the only option will be to load it through the chunking; later on, in the tutorial 'chunking_introduction', we will introduce this concept.

As the size of the data is not too big here, we can continue without any problem. But let's first have a look to the actual size and then how it impacts the memory once loaded into it.

In [ ]:
import numpy as np

In [ ]:
print(f'{np.round(ds.lai.nbytes / 1024**3, 2)} GB') # all the data are automatically loaded into memory as NumpyArray once they are accessed.

In [ ]:
ds.lai.data

As other datasets have dimensions named according to the more common triad lat,lon,time a renomination is needed.

## Selection methods

As underneath DataArrays are Numpy Array objects (that implement the standard Python x[obj] (x: array, obj: int,slice) syntax). Their data can be accessed through the same approach of numpy indexing.

In [ ]:
ds.lai[0,100,100].load()

As it is not easy to remember the order of dimensions, Xarray really helps by making it possible to select the position using names:

- `.isel` -> selection based on positional index
- `.sel`  -> selection based on coordinate values

In [ ]:
ds.lai.isel(time=0, latitude=100, longitude=100)

The more common way to select a point is through the labeled coordinate using the `.sel` method.

Time is easy to be used as there is a 1 to 1 correspondence with values in the index, float values are not that easy to be used and a small discrepancy can make a big difference in terms of results.


Coordinates are always affected by precision issues; the best option to quickly get a point over the coordinates is to set the sampling method (method='') that will search for the closest point according to the specified one.

Options for the method are:
- pad / **f**fill: propagate last valid index value forward
- backfill / **b**fill: propagate next valid index value backward
- nearest: use nearest valid index value

Another important parameter that can be set is the tolerance that specifies the distance between the requested and the target (so that abs(index\[indexer] - target) <= tolerance) from [documentation](https://xarray.pydata.org/en/v0.17.0/generated/xarray.DataArray.sel.html#:~:text=xarray.DataArray.sel%20%C2%B6%20DataArray.sel%28indexers%3DNone%2C%20method%3DNone%2C%20tolerance%3DNone%2C%20drop%3DFalse%2C%20%2A%2Aindexers_kwargs%29%20%C2%B6,this%20method%20should%20use%20labels%20instead%20of%20integers.).

In [ ]:
ds.lai.sel(time=datetime(2020, 1, 8), method='nearest')

In [ ]:
ds.lai.sel(latitude=46.3, longitude=8.8, method='nearest').isel(time=0)

:::{warning}
To select a single real value without specifying a method, you would need to specify the exact encoded value; not the one you see when printed.
:::

In [ ]:
ds.lai.isel(longitude=100).longitude.values.item()

#### How does Xarray with Dask distribute data analysis?

When we use chunks with `Xarray`, the real computation is only done when needed or asked for, usually when invoking `compute()` or `load()` functions. Dask generates a **task graph** describing the computations to be done. When using [Dask Distributed](https://distributed.dask.org/en/stable/) a **Scheduler** distributes these tasks across several **Workers**.

![Xarray with dask](./static/dask-xarray-explained.png)

#### How does Dask scale and accelerate your data analysis?

[Dask proposes different abstractions to distribute your computation](https://docs.dask.org/en/stable/10-minutes-to-dask.html). In this _Dask Introduction_ section, we will focus on [Dask Array](https://docs.dask.org/en/stable/array.html) which is widely used in pangeo ecosystem as a back end of Xarray.

As shown in the [previous section](./chunking_introduction.ipynb) Dask Array is based on chunks.
Chunks of a Dask Array are well-known Numpy arrays. By transforming our big datasets to Dask Array, making use of chunk, a large array is handled as many smaller Numpy ones and we can compute each of these chunks independently.

![Dask and Numpy](https://examples.dask.org/_images/dask-array-black-text.svg)




## Plotting
   Plotting data can easily be obtained through matplotlib.pyplot back-end [matplotlib documentation](https://matplotlib.org/stable/index.html).

As the exercise is focused on an Area Of Interest, this can be obtained through a bounding box defined with slices.



Plot fires

In [ ]:
lai_aoi = ds.lai.sel(latitude=slice(65.5,25.5), longitude=slice(-20.5,20.5))
lai_aoi.sel(time=datetime(2020,6,23), method='nearest').plot()


:::{tip}
Have you noticed that latitudes are selected from the largest to the smallest values e.g. `46.5`, `44.5` while longitudes are selected from the smallest to the largest value e.g. `8.5`,`11.5`?
**The reason is that you need to use the same order as the corresponding DataArray**.
:::

## Masking

Not all values are valid and masking all those which are not in the valid range is necessary. Masking can be achieved through the method `DataSet|Array.where(cond, other)` or `xr.where(cond, x, y)`.

The difference consists in the possibility to specify the value in case the condition is positive or not; `DataSet|Array.where(cond, other)` only offer the possibility to define the false condition value (by default is set to np.NaN))

# Basic Maths and Stats

In [ ]:
# E.g. example scaling 

lai_aoi * 0.0001 + 1500


# Statistics and Aggregation
Calculate simple statistics:


In [ ]:
lai_aoi.min()
lai_aoi.max()
lai_aoi.mean()

Aggregate by month if the dataset spans multiple months:

In [ ]:
lai_monthly = lai_aoi.groupby(lai_aoi.time.dt.month).mean()
lai_monthly

## Masking

Not all values are valid and masking all those which are not in the valid range is necessary. Masking can be achieved through the method `DataSet|Array.where(cond, other)` or `xr.where(cond, x, y)`.

The difference consists in the possibility to specify the value in case the condition is positive or not; `DataSet|Array.where(cond, other)` only offer the possibility to define the false condition value (by default is set to np.NaN))

In [ ]:

lai_masked = lai_aoi.where((lai_aoi >= 1.5)) 
lai_masked

In [ ]:
lai_masked.isel(time=0).plot(cmap='viridis')
plt.title("Masked Leaf Area Index")
plt.show()


In [ ]:
mask = xr.where((lai_aoi > 1.5), 1, 0)
mask.isel(time=0).plot()
plt.title("Masked Leaf Area Index")
plt.show()


By inspecting any of the variables on the representation above, you'll see that each data array represents __about 85GiB of data__, so much more than the availabe memory on this notebook server, and even on the Dask Cluster we created above. But thanks to chunking, we can still analyze it!

Did you notice something on the Dask Dashboard when running the two previous cells?

We didn't 'compute' anything. We just built a Dask task graph with it's size indicated as count above, but did not ask Dask to return a result.

But the 'task Count' we see above is more than 6000 for just computing a mean on 36 temporal steps. This is too much.  If you have such case, to avoid unecessary operations, you can optimize the task using `dask.optimize`. 

Lets try to plot the dask graph before computation and understand what dask workers will do to compute the value we asked for. 

Calling compute or triggering it (via plot) on our Xarray object triggered the execution on Dask Cluster side. 

You should be able to see how Dask is working on Dask Dashboard. 

# Forest Fires Example Workflow

Search for a forest fire in Europe this last year

In [ ]:
ds.gfed_ba

In [ ]:
gwis = ds.gwis_ba
gwis

In [ ]:
polygon = gpd.GeoDataFrame(index=[0], crs="epsg:4326", geometry=[geometry.box(*bbox)])
min_lon, min_lat, max_lon, max_lat = polygon.total_bounds

offset = 0
zoom = 1

lat_start = gwis.latitude.sel(latitude=max_lat, method="nearest").item()   + offset*zoom
lat_stop  = gwis.latitude.sel(latitude=min_lat, method="nearest").item()   - offset*zoom
lon_start = gwis.longitude.sel(longitude=min_lon, method="nearest").item()  - offset*zoom  # better orientation
lon_stop  = gwis.longitude.sel(longitude=max_lon, method="nearest").item()  + offset*zoom


In [ ]:
## in SeasFire Datacube v2.0 the burned area variables would have the water bodies masked with ERA-5 land sea mask 
mask= ds['lsm'][:,:]
mask

Find the biggest forest fire during the last three years within that place

In [ ]:
gwis_aoi = gwis.sel(time=slice(datetime(2018,1,1),datetime(2021,1,1)), latitude=slice(lat_start, lat_stop),longitude=slice(lon_start, lon_stop))

In [ ]:
dateind_max_fire = gwis_aoi.sum(dim={'latitude','longitude'}).argmax(dim='time').compute().item()
fire_date = gwis_aoi.time[dateind_max_fire]
# where the sum in the plot is the highest

In [ ]:
date_max_fire = gwis_aoi.isel(time=dateind_max_fire)
date_max_fire

In [ ]:
# gwis_all=gwis.resample(time="1Y").sum()
gwis_all=date_max_fire
gwis_all= gwis_all.where(mask)
gwis_all.plot()


In [ ]:
# import cartopy
# import matplotlib.pyplot as plt
# import cartopy.crs as ccrs



# da = gwis_all

# fig, ax = plt.subplots(
#     figsize=(8, 6),
#     subplot_kw={"projection": ccrs.PlateCarree()}
# )

# da.plot(
#     ax=ax,
#     transform=ccrs.PlateCarree(),
#     cmap="viridis",
#     cbar_kwargs={"label": "T₂ₘ (K)"}
# )

# ax.coastlines(resolution="10m", color="black", linewidth=1)
# ax.add_feature(cartopy.feature.BORDERS, linewidth=0.5)

# ax.set_extent([lon_start, lon_stop, lat_stop, lat_start], crs=ccrs.PlateCarree())

# plt.show()

In [ ]:
import hvplot.xarray

# Plot it interactively
gwis_all.hvplot(
    x='longitude',
    y='latitude',
    cmap='viridis',
    colorbar=True,
    frame_width=600,
    frame_height=400,
    geo=True,
    tiles='OSM'
)


In [ ]:
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature


# lat_start=41
# lat_stop=41.5
# lon_start=-7.7
# lon_stop=-7.5

# lat_slice=slice(41.5,41)
# lon_slice=slice(-7.7,-7.5)

# fires = gwis_all.sel(latitude=lat_slice,longitude=lon_slice)
fire_date_t = pd.to_datetime(fire_date.values.item())

week_before = (fire_date_t - timedelta(days=7))
week_after = (fire_date_t + timedelta(days=7))

In [ ]:
print(week_before.date(), "---" , week_after.date())

In [ ]:
# from pystac_client import Client as pystac_client
# from odc.stac import configure_rio, stac_load


# Open a catalog
# catalog = pystac_client.open("https://earth-search.aws.element84.com/v1")
# time_start = "2020-07-01"
# time_end = "2020-07-28"
# time_range = time_start+"/"+time_end

In [ ]:
epsg

In [ ]:
index_name = 'NBR'

bandnames_dict = {
    'nir': 'nir',
    'swir22': 'swir22'
}
crs = "epsg:"+ str(epsg)  # projection on which the data will be projected

# km2deg = 1.0 / 111
# x, y = (23.9983519, 37.7351433)  # Center point of a query
# r = 4 * km2deg  
# bbox = (x - r, y - r, x + r, y + r)
# zoom = 1


# Normalised Burn Ratio, Lopez Garcia 1991
def calc_nbr(ds):
    return (ds.nir - ds.swir22) / (ds.nir + ds.swir22)

index_dict = {'NBR': calc_nbr}
index_dict

In [ ]:
catalog = pystac_client.open("https://earth-search.aws.element84.com/v1")


In [ ]:
zoom=1/2
chunk={}


In [ ]:
week_before_start = (week_before - timedelta(days=30))
time_range = str(week_before_start.date()) + "/" + str(week_before.date())

query1 = catalog.search(
    collections=["sentinel-2-l2a"], datetime=time_range, limit=100,
    bbox=bbox, query={"eo:cloud_cover": {"lt": 0.5}}
)

items = list(query1.get_items())
print(f"Found: {len(items):d} datasets")

items_pre = min(items, key=lambda item: item.properties["eo:cloud_cover"])

prefire_ds = stac_load(
    [items_pre],
    bands=("nir", "swir22"),
    # crs=crs,
    # resolution=  10*zoom,
    chunks=chunk,  # <-- use Dask
    groupby="datetime",
    bbox=bbox,
)
prefire_ds

In [ ]:
week_after_end = (week_after + timedelta(days=30))
time_range = str(week_after.date()) + "/" + str(week_after_end.date())

query2 = catalog.search(
    collections=["sentinel-2-l2a"], datetime=time_range, limit=100,
    bbox=bbox, query={"eo:cloud_cover": {"lt": 0.5}}
)

items = list(query2.get_items())
print(f"Found: {len(items):d} datasets")

items_post = min(items, key=lambda item: item.properties["eo:cloud_cover"])

postfire_ds = stac_load(
    [items_post],
    bands=("nir", "swir22"),
    # crs=crs,
    # resolution=10 * zoom,
    chunks=chunk,  # <-- use Dask
    groupby="datetime",
    bbox=bbox,
)
postfire_ds

In [ ]:
# Rename bands in dataset to use simple names 
bands_to_rename = {
    a: b for a, b in bandnames_dict.items() if a in prefire_ds.variables
}

# prefire
prefire_ds[index_name] = index_dict[index_name](prefire_ds.rename(bands_to_rename) / 10000.0)

# postfire
postfire_ds[index_name] = index_dict[index_name](postfire_ds.rename(bands_to_rename) / 10000.0)


In [ ]:
# calculate delta NBR
prefire_burnratio = prefire_ds.NBR.isel(time=0)
postfire_burnratio = postfire_ds.NBR.isel(time=0)

delta_NBR = prefire_burnratio - postfire_burnratio

dnbr_dataset = delta_NBR.to_dataset(name='delta_NBR').persist()

In [ ]:
dnbr_dataset
delta_NBR

In [ ]:
fig = plt.figure(1, figsize=[7, 10])

# We're using cartopy and are plotting in PlateCarree projection 
# (see documentation on cartopy)
ax = plt.subplot(1, 1, 1, projection=ccrs.PlateCarree())
#ax.set_extent([-180, 180, -70, 70], crs=ccrs.PlateCarree()) # lon1 lon2 lat1 lat2
ax.coastlines(resolution='10m')
ax.gridlines(draw_labels=True)

# We need to project our data to the new Orthographic projection and for this we use `transform`.
# we set the original data projection in transform (here Mercator)
prefire_burnratio.plot(ax=ax, transform=ccrs.epsg(prefire_burnratio.spatial_ref.values), cmap='RdBu_r',
                       cbar_kwargs={'orientation':'horizontal','shrink':0.95})

# One way to customize your title
plt.title( pd.to_datetime(prefire_burnratio.time.values.item()).strftime("%d %B %Y"), fontsize=18)

In [ ]:
fig = plt.figure(1, figsize=[7, 9])

# We're using cartopy and are plotting in PlateCarree projection 
# (see documentation on cartopy)
ax = plt.subplot(1, 1, 1, projection=ccrs.PlateCarree())
#ax.set_extent([-180, 180, -70, 70], crs=ccrs.PlateCarree()) # lon1 lon2 lat1 lat2
ax.coastlines(resolution='10m')
ax.gridlines(draw_labels=True)

# We need to project our data to the new Orthographic projection and for this we use `transform`.
# we set the original data projection in transform (here Mercator)
postfire_burnratio.plot(ax=ax, transform=ccrs.epsg(postfire_burnratio.spatial_ref.values), cmap='RdBu_r',
                        cbar_kwargs={'orientation':'horizontal','shrink':0.95})

# One way to customize your title
plt.title( pd.to_datetime(postfire_burnratio.time.values.item()).strftime("%d %B %Y"), fontsize=18)

In [ ]:
fig = plt.figure(1, figsize=[7, 10])

# We're using cartopy and are plotting in PlateCarree projection 
# (see documentation on cartopy)
ax = plt.subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.coastlines(resolution='10m')
ax.gridlines(draw_labels=True)

# We need to project our data to the new Orthographic projection and for this we use `transform`.
# we set the original data projection in transform (here Mercator)
dnbr_dataset.delta_NBR.plot(ax=ax, transform=ccrs.epsg(dnbr_dataset.delta_NBR.spatial_ref.values), cmap='RdBu_r',
                            cbar_kwargs={'orientation':'horizontal','shrink':0.95})

# One way to customize your title
plt.title( "Delta NBR", fontsize=18)

https://un-spider.org/advisory-support/recommended-practices/recommended-practice-burn-severity/in-detail/normalized-burn-ratio

![img](https://un-spider.org/sites/default/files/table+legend.PNG)

In [ ]:
BURN_THRESH = 0.27
burn_mask = dnbr_dataset.delta_NBR > BURN_THRESH           # True/False mask, same shape as raster
burn_mask.plot()

In [ ]:
dx, dy = dnbr_dataset.delta_NBR.rio.resolution()
pixel_area_ha = abs(dx * dy) / 1e4       # 10m × 10m  → 0.01 ha
pixel_area_ha

pixels_burned   = burn_mask.sum().compute().item()   # integer number of burned pixels
burned_area_ha  = pixels_burned * pixel_area_ha

print(f"Pixels burned : {pixels_burned:,d}")
print(f"Burned area   : {burned_area_ha:,.2f} ha")
print(f"Actual Burned Area : {gwis_all.sum().compute():,.2f}, ha")

In [ ]:
dnbr_dataset['burned_ha_mask'] = burn_mask

In [ ]:
dnbr_dataset = dnbr_dataset.persist()
gwis_all = gwis_all.persist()


# r_dnbr_dataset = dnbr_dataset.rename({'x': 'lon', 'y': 'lat'})
# r_gwis_all = gwis_all.rename({'longitude': 'lon', 'latitude': 'lat'})

In [ ]:
gwis_all = gwis_all.rio.write_crs(ds.rio.crs)

Plot is off because of bad projection (curcilinear to rectilinear) but we can see that generally the fires are in the north-west/north-east regions with two distinct occurances

In [ ]:
import hvplot.xarray

gwis_all_reprojected = gwis_all.rio.reproject(dnbr_dataset.delta_NBR.rio.crs)

dnbr_plot = dnbr_dataset.delta_NBR.hvplot(
    width=700,
    height=700,
    title='dNBR (10 m) with GWIS overlay',
    alpha=1.0
)

# Plot the reprojected coarse dataset as transparent overlay
gwis_plot = gwis_all_reprojected.hvplot(
    cmap='Reds',
    alpha=0.3,
    clim=(0, gwis_all.max().compute().item())
)

# Combine them interactively
combined_plot = dnbr_plot * gwis_plot

combined_plot


# Saving Your Work

xrlint and validate your cube...

In [ ]:
import xrlint.all as xrl

linter = xrl.new_linter("recommended")
linter.validate(dnbr_dataset)


In [ ]:
# Assign dataset-level attributes
dnbr_dataset.attrs.update({
    'title': 'Delta NBR and Burned Area Mask Dataset',
    'history': 'Created by reprojecting and aligning datasets for fire severity analysis',
    'Conventions': 'CF-1.7'
})


# Assign variable-level attributes for delta_NBR
dnbr_dataset.delta_NBR.attrs.update({
    'institution': 'Lampata',
    'source': 'Sentinel-2 imagery; processed with open-source dNBR code, element84...',
    'references': 'https://example.com/ref',
    'comment': 'dNBR values represent change in vegetation severity post-fire',
    'standard_name': 'difference_normalized_burn_ratio',
    'long_name': 'Differenced Normalized Burn Ratio (dNBR)',
    'units': 'm'
})

# Example for burned_ha_mask data variable
dnbr_dataset.burned_ha_mask.attrs.update({
    'standard_name': 'burned_area_mask',
    'long_name': 'Burned Area Mask in Hectares',
    'units': 'hectares',
    'institution': 'Your Institution Name',
    'source': 'Derived from wildfire impact analysis',
    'references': 'https://example.com/ref',
    'comment': 'Burned area mask showing presence of burned areas'
})


In [ ]:
from xcube.core.verify import assert_cube

assert_cube(dnbr_dataset)  # raises ValueError if it's not xcube-valid


In [ ]:
# add time
dnbr_dataset = dnbr_dataset.expand_dims(time=[postfire_ds.time.isel(time=0).values])
dnbr_dataset

Chunk Data for Better Usability

In [ ]:
dnbr_dataset = dnbr_dataset.chunk({"time": 1, "y": 1000, "x": 1000})


In [ ]:
print(type(dnbr_dataset.burned_ha_mask.data))

In [ ]:
import os
save_at_folder = './wildfires'
if not os.path.exists(save_at_folder):
    os.makedirs(save_at_folder)

# Define the output path within your notebook folder
output_path = os.path.join(save_at_folder, "dnbr_dataset.zarr")

# save
dnbr_dataset.to_zarr(output_path, mode="w")